<a href="https://colab.research.google.com/github/harini-si/saidl-assignment/blob/main/semi_supervised_saidl_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader,Dataset
from torchvision import datasets
import torch.optim as optim
from PIL import Image
import numpy as np

In [ ]:
#loading the data
train_data=datasets.STL10(root="./data",split='train',download=True,transform=ToTensor())
test_data=datasets.STL10(root="./data",split='test',download=True,transform=ToTensor())
train_loader=DataLoader(train_data,shuffle=True,batch_size=32)
test_loader=DataLoader(test_data,shuffle=True,batch_size=32)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import torch.backends.cudnn as cudnn
device = 'cuda'


In [ ]:
torch.cuda.empty_cache()

In [ ]:
#defining the network
class block(nn.Module):
  def __init__(self,in_channels,out_channels,stride=1):
    super(block,self).__init__()
    self.flag = False
    self.conv1=nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=stride,padding=1)
    self.bn1=nn.BatchNorm2d(out_channels)
    self.conv2=nn.Conv2d(out_channels,out_channels,kernel_size=3,padding=1)
    self.bn2=nn.BatchNorm2d(out_channels)
    self.relu=nn.ReLU()
    if stride != 1 or in_channels != out_channels:
      self.flag = True
      self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride,bias=False),
                nn.BatchNorm2d(out_channels)
            )
    else:
      self.shortcut = nn.Sequential()
  def forward(self, x):
    out = self.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    if self.flag:
      x = self.shortcut(x)
    out += x
    out = self.relu(out)
    return out
    
class ResNet(nn.Module):
  def __init__(self):
    super(ResNet,self).__init__()
    self.conv1= nn.Conv2d(3,64,kernel_size= 3,stride=1,padding=1,bias=False)
   
    
    self.bn1=nn.BatchNorm2d(64)
    self.conv2a=block(64,64)
    self.conv2b=block(64,64,2)
    self.conv3a=block(64,64)
    self.conv3b=block(64,128,2)
    
    self.conv4a=block(128,128)
    self.conv4b=block(128,256,2)
    
    
    self.conv5a=block(256,256)
    self.conv5b=block(256,512,2)
    #self.conv6a=block(512,512)
    #self.conv6b=block(512,512,1)
    self.relu = nn.ReLU()
    self.dropout=nn.Dropout2d(0.6)
    self.avg_pool=nn.AvgPool2d(kernel_size=4)
    self.linear = nn.Linear(512 , 10)
  def forward(self,x):
    x=self.relu(self.bn1(self.conv1(x)))
    
    #x=F.max_pool2d(x,2,2)
    x=self.conv2a(x)
    x=self.dropout(x)
    x=self.conv2b(x)
    #x=F.max_pool2d(x,2,2)
    x=self.conv3a(x)
    x=self.dropout(x)
    x=self.conv3b(x)
    
    #x=F.max_pool2d(x,2,2)
    
    x=self.conv4a(x)
    x=self.dropout(x)
    x=self.conv4b(x)
    #x=F.max_pool2d(x,2,2)
    x=self.conv5a(x)
    x=self.dropout(x)
    
    x=self.conv5b(x)
    #x=self.conv6a(x)
    #x=self.dropout(x)
    
    #x=self.conv6b(x)
    x=self.avg_pool(x)
  
    x=x.view(x.size(0), -1)
    x=self.linear(x)
   
    return x

In [ ]:
#hyperparameters
model=ResNet().to('cuda')
learning_rate=0.001
batch_size=32
num_epochs=20

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
 
# Applying it to our net
model.apply(init_weights)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2a): block(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (shortcut): Sequential()
  )
  (conv2b): block(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (shortcut): Sequential

In [ ]:
#loss and optimizer
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999),weight_decay=1e-5, eps=1e-08, amsgrad=False)


In [ ]:
#def test loop and supervised training
def test_loop(dataloader, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X=X.to('cuda')
            y=y.to('cuda')
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    Accuracy=100*correct
    return Accuracy
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
   
    correct=0
    total=0
    for i, (images, labels) in enumerate(train_loader):
      
        images = images.to('cuda')
        labels = labels.to('cuda')

        
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
       
        
    
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
      

        if (i+1) % 20 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
      
    
    accu= 100 * correct / len(train_data)
    test_accu=test_loop(test_loader,model)
    print(f'Epoch [{epoch+1}/{num_epochs}], train_accuracy: {accu:.4f},test_accuracy: {test_accu:.4f} %')   


In [ ]:

PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
unlabeled=datasets.STL10(root="./data",split='unlabeled',download=True,transform=ToTensor())

unlabeled1=torch.utils.data.Subset(unlabeled,range(0,int (len(unlabeled)/5), 2))
unlabelled_loader1=DataLoader(unlabeled1,shuffle=True,batch_size=32)

Files already downloaded and verified


In [ ]:
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [ ]:
#threshold for p labels
num_epochs1=10
threshold=0.95

In [ ]:
def generate_labels(unlabelled_loader1,threshold):
  model.eval()
  x=0
  plabels=[]
  image=[]
  with torch.no_grad():
    for i, (images,labels) in enumerate (unlabelled_loader1):
      images=images.to('cuda')
      labels=labels.to('cuda')
    
      outputs=model(images)
      outputs=F.softmax(outputs,dim=1)
      if((outputs.max()>threshold) and images!=[]):
      
        predicted=outputs.max(1)[1]
        plabels.append(predicted)
        image.append(images)
      
      #image.append([images,predicted])
      
    return plabels,image
pl,imgs=generate_labels(unlabelled_loader1,threshold)
print(len(imgs))

313


In [ ]:
dataTensor = torch.cat(pl)
print(dataTensor)
dataTensor.to('cuda')
imgTensor=torch.cat(imgs)
print(len(imgTensor))
imgTensor.to('cuda')
i=[imgTensor]
p=[dataTensor]

tensor([8, 6, 7,  ..., 5, 6, 6], device='cuda:0')
10000


In [ ]:
class MyDataset(Dataset):
    def __init__(self, data,targets, transform=None):
        self.data = i[0]
        self.targets = p[0]
       
        
    def __getitem__(self, index):
        x = self.data[index]
        y=self.targets[index]
     

     
        
        return x,y
    
    def __len__(self):
        return len(self.data)



transform = transforms.Compose([transforms.Resize(96), transforms.ToTensor()])
dataset = MyDataset(i,p )
print(dataset[0][0].shape)



torch.Size([3, 96, 96])


In [ ]:
x=[]
y=[]
for i,(images,labels) in enumerate(train_loader):
  images=images.to(device)
  labels=labels.to(device)
  x.append(images)
  y.append(labels)

In [ ]:
train_x=torch.cat(x)
train_y=torch.cat(y)
type(train_y)

torch.Tensor

In [ ]:
class tDataset(Dataset):
    def __init__(self, data,targets, transform=None):
        self.data = train_x
        self.targets = train_y
        print(len(self.data))
        
        self.transform = transform
        
    def __getitem__(self, i):
        x = self.data[i]
        y=self.targets[i]
        return x,y
    
    def __len__(self):
        return len(self.data)
data = tDataset(train_x,train_y)
len(data)


5000


5000

In [ ]:
concatDataset=torch.utils.data.ConcatDataset([dataset,data])
loader = DataLoader(concatDataset, batch_size=32,shuffle=True)
len(concatDataset)

15000

In [ ]:
model=ResNet().to(device)

loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999),weight_decay=1e-5, eps=1e-08, amsgrad=False)

In [ ]:
n_total_steps = len(loader)
for epoch in range(num_epochs1):
   
    correct=0
    total=0
    for i, (images, labels) in enumerate(loader):
      
    
        images = images.to(device)
        labels = labels.to(device,dtype=torch.int64)
      

        
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
       
        
    
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
      

        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs1}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
      
    
    accu= 100 * correct / len(concatDataset)
    test_accu=test_loop(test_loader,model)
    print(f'Epoch [{epoch+1}/{num_epochs1}], train_accuracy: {accu:.4f},test_accuracy: {test_accu:.4f} %')   